In [1]:
import os

if False:
    # Set environment variables
    falcor_dir = "D://NeuralFalcor//neuralrender2//build//windows-vs2022//bin//Release//"  # Replace with the correct path
    os.environ["FALCOR_DIR"] = falcor_dir
    os.environ["PATH"] = f"{falcor_dir};{os.environ['PATH']}"
    os.environ["PYTHONPATH"] = f"{falcor_dir}python;{os.environ.get('PYTHONPATH', '')}"

    # Confirm the environment variables are set
    print("FALCOR_DIR:", os.environ["FALCOR_DIR"])
    print("PATH:", os.environ["PATH"])
    print("PYTHONPATH:", os.environ["PYTHONPATH"])

In [2]:
import torch
import falcor
import time
import numpy as np
import pyexr as exr
import sys
import os
import dataclasses
import datetime
import glob
import argparse

sys.path.append(os.path.join(os.path.abspath(''), ".."))
import common
import material_utils
from loss import compute_render_loss_L1, compute_render_loss_L2


import common
import os
from falcor import Camera, float3, uint2
import copy
import numpy as np
from typing import List, Tuple

import torch
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

import json
try:
	import tinycudann as tcnn
except ImportError:
	print("This sample requires the tiny-cuda-nn extension for PyTorch.")
	print("You can install it by running:")
	print("============================================================")
	print("tiny-cuda-nn$ cd bindings/torch")
	print("tiny-cuda-nn/bindings/torch$ python setup.py install")
	print("============================================================")
	sys.exit()

d:\miniconda\envs\falcor-pytorch\lib\site-packages\tinycudann\modules.py:53: UserWarning: tinycudann was built for lower compute capability (86) than the system's (89). Performance may be suboptimal.
  warnings.warn(f"tinycudann was built for lower compute capability ({cc}) than the system's ({system_compute_capability}). Performance may be suboptimal.")


In [3]:
import yaml

class ExperimentParams:
    def __init__(self):
        # Load the configuration from YAML file
        with open('config.yaml', 'r') as file:
            config = yaml.safe_load(file)['ExperimentParams']

        self.MODELS_PATH = config.get('MODELS_PATH')

params = ExperimentParams()
print(params.MODELS_PATH)  

E:/Models/


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as patches

def float3tonumpy(v):
    return np.array([v.x, v.y, v.z])

def numpytofloat3(n):
    return float3(*np.copy(n))

def numpytouint2(n):
    return uint2(*np.copy(n))


class ExperimentParams:
    #render_width = 256 # random
    #render_height = 256
    render_width = 1280 # random
    render_height = 720

    enable_MIS = False 
    enable_sky_learning = True # for example we could disable it and compare NIRC with NRC

    relative_error = False

    USE_GBUFFER = True
    relative_error_eps = 0.001
    

    RECOMPUTE_LOSS = {"NIRC": False, "NIRC_EM": False, "SH": False, "VMF": False, "NCV": False}

    GRADIENT_CLIP = 0.001
    SKIP_TENSOR_CHECK = True # disable if you have problems with NaNs, Inf or just 0 loss
    ADAM_BETAS = [0.98, 0.9999]
    ADAM_LR = 0.01
    num_training_frames = 150 # we gotta use the same number of training frames for all models to make conduct fair experiments!. USE a BIIG number for the final experiments (4k? 5k?)

    # MODELS_PATH = "E:/Models/" # we can just put it in the same folder as the script?
    MODELS_PATH = params.MODELS_PATH

    is_training = False



gparams = ExperimentParams()



class ValidationPoint:
    xn: float
    yn: float

    x: int
    y: int

    id: int
    
    position: np.ndarray
    target: np.ndarray
    normal: np.ndarray

    is_init: bool

    def __init__(self, xn: float, yn: float):
        self.xn = xn
        self.yn = yn

        self.x = int(gparams.render_width*xn)
        self.y = int(gparams.render_height*yn)

        self.id = self.y*gparams.render_width+self.x

        self.is_init = False


    def init_surface_data(self, p: np.ndarray, t: np.ndarray, n: np.ndarray):
        self.is_init = True

        self.position = p
        self.target = t
        self.n = n




class SceneConfig:
    model_path: str
    camera_position: np.ndarray
    camera_target: np.ndarray
    tonemapper_exposure: float
    focal_length: float
    
    validation_points: List[ValidationPoint]
    emissive_factor: float = 1.0
    

    def __init__(self, model_path: str, camera_position: Tuple[float, float, float], camera_target: Tuple[float, float, float], tonemapper_exposure: float = 0, focal_length: float = None, selected_points: List[Tuple[float, float]] = [], 
                 up: Tuple[float, float, float] = [0.0, 1.0, 0.0], lr_factor: float = 1.0, epochs: int = 4000, roughness: float = 1.0, var_est_steps: int = 1000):
        self.model_path = gparams.MODELS_PATH + model_path
        self.model_name = os.path.dirname(model_path)
        self.camera_position = np.array(camera_position)
        self.camera_target = np.array(camera_target)
        self.epochs = epochs
        self.var_est_steps = var_est_steps
        self.tonemapper_exposure = tonemapper_exposure
        self.roughness = roughness
        self.lr_factor = lr_factor
        self.checkpoint_dir = os.path.join('checkpoints', self.model_name)
        os.makedirs(self.checkpoint_dir, exist_ok=True)

        
        self.focal_length = focal_length
        self.validation_points = []
        self.up = up

        for p in selected_points:
            self.validation_points.append(ValidationPoint(xn=p[0], yn=p[1]))

    def camera_position_f3(self) -> float3:
        return numpytofloat3(self.camera_position)

    def camera_target_f3(self) -> float3:
        return numpytofloat3(self.camera_target)

    def __repr__(self) -> str:
        return f"SceneConfig(model_path={self.model_path}, camera_position={self.camera_position.tolist()}, camera_target={self.camera_target.tolist()}, tonemapper_exposure={self.tonemapper_exposure}, focal_length={self.focal_length})"

    def add_validation_point(self, p: Tuple[float, float]):
        self.validation_points.append(ValidationPoint(xn=p[0], yn=p[1]))

    def setup_validation_points(self, mlData):
        positions = mlData["worldpos"] 
        normals = mlData["normal"]

        for i in range(len(self.validation_points)):
            p = self.validation_points[i]
            pos = positions[p.id].cpu().numpy() # we should get it from the gpu torch in some way 
            normal = normals[p.id].cpu().numpy()

            p.init_surface_data(p = pos+normal*0.00001, t = pos+normal+normal*0.00001, n = normal)
            
    def prepare_scene(self, scene, debugPointID=None):
        scene.roughnessMultiplier = self.roughness

        if debugPointID == None:
            scene.camera.position = numpytofloat3(self.camera_position)
            scene.camera.target = numpytofloat3(self.camera_target)
            if self.focal_length != None:
                scene.camera.focalLength = self.focal_length
            scene.camera.useHemisphericalCamera = False
            scene.camera.up = numpytofloat3(self.up)
        else:
            vp = self.validation_points[debugPointID]
            scene.camera.useHemisphericalCamera = True
            scene.camera.position = numpytofloat3(vp.position)
            scene.camera.target = numpytofloat3(vp.target)
            scene.camera.up = numpytofloat3(self.up)
            assert(vp.is_init)

# some specular scenes? we could just import the specular sponza
scenes = {
    "CornellBox": SceneConfig("CornellBox/cornell_box.pyscene", camera_position=[0, 0.28, 0.6], camera_target=[0, 0.28, 0], selected_points=[[0.35, 0.5], [0.65, 0.4]], epochs=2000),
    "CornellBoxEnv": SceneConfig("CornellBoxEnv/cornell_box_env.pyscene", camera_position=[0, 0.28, 0.6], camera_target=[0, 0.28, 0], selected_points=[[0.35, 0.5], [25.0/256.0, 150.0/256.0]]),
    "EnvDebug": SceneConfig("EnvDebug/env_debug.pyscene", camera_position=[0, 0.28, 0.6], camera_target=[0, 0.28, 0], selected_points=[[0.35, 0.5], [0.65, 0.4]]),
    #"CornellBox": SceneConfig("CornellBox/cornell_box.pyscene", camera_position=[0, 0.28, 1.2], camera_target=[0, 0.28, 0], selected_points=[[0.35, 0.5], [0.65, 0.4]]),
    "Bistro" : SceneConfig("Bistro/BistroExterior.pyscene", camera_position=[-29.195, 5.145, -8.768], camera_target=[-28.212, 5.137, -8.586], selected_points=[[400.0/1280, 400.0/720], [410/1200.0, 200.0/720], [800/1280.0, 600/720.0]]),
    "CountryKitchen": SceneConfig("CountryKitchen/Country-Kitchen.gltf", camera_position=[1.456, 1.509, 1.602], camera_target=[0.678, 1.471, 0.974], selected_points=[[0.35, 0.5], [0.65, 0.4]], lr_factor=0.5, epochs=2000, var_est_steps=3000),
    "SanMiguel": SceneConfig("SanMiguel/san-miguel.pyscene", camera_position=[22.2190, 3.3300, 6.7120], camera_target=[21.5799, 3.3054, 5.9432], selected_points=[[0.35, 0.5], [0.65, 0.4]], focal_length=17.750),
    "TheWhiteRoomCycles": SceneConfig("TheWhiteRoomCycles/the-white-room_0001.gltf", camera_position=[2.781, 1.247, 5.251], camera_target=[2.151, 1.195, 4.477], selected_points=[[0.35, 0.5], [0.65, 0.4]], lr_factor=0.25),
    "Sponza": SceneConfig("Sponza/Sponza.pyscene", camera_position=[8.084, 1.708, 0.827], camera_target=[7.091, 1.684, 0.715], selected_points=[[0.5, 0.1], [0.65, 0.2]], focal_length=16.750),
    "SponzaSpecular": SceneConfig("SponzaSpecular/SponzaSpecular.pyscene", camera_position=[-9.0311, 1.1590, -1.2149], camera_target=[-8.0785, 1.0369, -0.9365], selected_points=[[0.5, 0.8], [0.15, 0.5]], epochs=2000, roughness=0.25) 
}


## Scene Select

In [5]:
scene_cfg = scenes["CornellBox"]

In [6]:
CUR_DIR = os.path.abspath('')
sys.path.append(os.path.join(CUR_DIR, ".."))


output_dir = CUR_DIR + "/results/"


device_id = 0
testbed = common.create_testbed([gparams.render_width, gparams.render_height])
device = testbed.device


# Load the reference scene.
ref_scene = common.load_scene(
    testbed,
    scene_cfg.model_path,
    gparams.render_width / gparams.render_height,
)


# I don't know why but useAnalyticLights and useEnvLight have False value if we start the renderer from the python side. haven't managed to found the root of this bizarre behaviour. it must be a bug -> TO DO: MAKE SURE and REPORT ABOUT IT!!
ref_scene.renderSettings = falcor.SceneRenderSettings(useEnvLight=True, useAnalyticLights=True, useEmissiveLights=True, useGridVolumes=True)


# init structure buffers that we need for ML side
# color = albedo+specular reflectance
field_types = {"radiance": "float3", "dir": "float3", "thp": "float3", "worldpos": "float3", "normal": "float3", "color": "float3", "dradiance": "float3", "view": "float3", "roughness": "float", "pdf": "float"}
ml_data = device.create_structured_buffer(
    struct_size = 12*8+4*2,
    element_count=gparams.render_width*gparams.render_height,
    bind_flags=falcor.ResourceBindFlags.ShaderResource  
    | falcor.ResourceBindFlags.UnorderedAccess
    | falcor.ResourceBindFlags.Shared
)


rays_fields = {"worldpos": "float3", "dir": "float3"}
ml_rays_data = device.create_structured_buffer(
    struct_size = 12*2,
    element_count=gparams.render_width*gparams.render_height,
    bind_flags=falcor.ResourceBindFlags.ShaderResource  
    | falcor.ResourceBindFlags.UnorderedAccess
    | falcor.ResourceBindFlags.Shared
)


device.render_context.wait_for_falcor()

In [7]:
render_graph = testbed.create_render_graph("StandardPathTracer")

# Create the PathTracer pass.
path_tracer_pass = render_graph.create_pass(
    "PathTracer",
    "PathTracer",
    {
        "samplesPerPixel": 1, "useSER": False, "useMIS": gparams.enable_MIS, "disableCaustics": True
    }
)


primary_render_pass_name = "GBufferRT" if gparams.USE_GBUFFER else "VBufferRT"
if  gparams.USE_GBUFFER:
    primary_render_pass = render_graph.create_pass(
        primary_render_pass_name,
        primary_render_pass_name,
        {
            "samplePattern": "Center",
            "sampleCount": 1,
            "useAlphaTest": True
        }
    )
    render_graph.mark_output(primary_render_pass_name+".vbuffercache")
    render_graph.mark_output(primary_render_pass_name+".brdf")
else:
    # Create the VBufferRT pass.
    primary_render_pass = render_graph.create_pass(
        primary_render_pass_name,
        primary_render_pass_name,
        {
            "samplePattern": "Center",
            "sampleCount": 1,
            "useAlphaTest": True
        }
    )

AccumulatePass = render_graph.createPass("AccumulatePass", "AccumulatePass", {'enabled': True, 'precisionMode': 'Single'})
ToneMapper = render_graph.createPass("ToneMapper", "ToneMapper", {'autoExposure': False, 'exposureCompensation': 0.0, 'outputFormat': 'RGBA32Float' }) 



# Add edges to connect the passes.

render_graph.add_edge(primary_render_pass_name+".vbuffer", "PathTracer.vbuffer")


render_graph.add_edge(primary_render_pass_name+".viewW", "PathTracer.viewW")
render_graph.add_edge(primary_render_pass_name+".mvec", "PathTracer.mvec")

render_graph.addEdge("PathTracer.color", "AccumulatePass.input")
render_graph.addEdge("AccumulatePass.output", "ToneMapper.src")


# Mark the output of the PathTracer pass.
render_graph.markOutput("ToneMapper.dst")
render_graph.mark_output("AccumulatePass.output")
render_graph.mark_output("PathTracer.color")

# Assign the configured render graph to the testbed.
testbed.render_graph = render_graph

path_tracer_pass.mlData = ml_data
primary_render_pass.mlRaysData = ml_rays_data


In [8]:
scene_cfg.prepare_scene(ref_scene)

In [9]:
import matplotlib.pyplot as plt 


def adjust_gamma(image, gamma=2.2):
	return (image )**(1 / gamma)


def frameRender(num_samples=1024, vis = True, tonemapped=False,  directEmissive=True, directSky=True):
    # may take some time to recompile the shaders becase of changed defines
    AccumulatePass.reset()
    AccumulatePass.enabled = True
    primary_render_pass.sampleCount = 16
    
    path_tracer_pass.useMIS = True
    path_tracer_pass.mlTraining = False
    path_tracer_pass.directEmissive = directEmissive
    path_tracer_pass.directSky = directSky

    for _ in range(num_samples):
        testbed.frame()

    if tonemapped:
        img = testbed.render_graph.get_output("ToneMapper.dst").to_numpy()[:, :, :3]
        img = adjust_gamma(img)
    else:
        img = testbed.render_graph.get_output("AccumulatePass.output").to_numpy()[:, :, :3]
    if not vis:
        return img


    plt.imshow(img)

    path_tracer_pass.directEmissive = True
    path_tracer_pass.directSky = True
    path_tracer_pass.useMIS = gparams.enable_MIS 
    primary_render_pass.sampleCount = 1
    AccumulatePass.enabled = False
    return img

In [10]:
def trainFrame():
    path_tracer_pass.useMIS = False
    path_tracer_pass.indirectSky = gparams.enable_sky_learning
    path_tracer_pass.mlTraining = True
    testbed.frame()
    path_tracer_pass.useMIS = True
    path_tracer_pass.indirectSky = True
    path_tracer_pass.mlTraining = False
        
def setupBRDFCache():
    path_tracer_pass.useMIS = False
    path_tracer_pass.indirectSky = gparams.enable_sky_learning
    path_tracer_pass.mlTraining = True
    primary_render_pass.cacheVisibility = True
    primary_render_pass.brdfRender = False
    testbed.frame()
    primary_render_pass.brdfRender = False
    primary_render_pass.cacheVisibility = False
    path_tracer_pass.useMIS = True
    path_tracer_pass.indirectSky = True
    path_tracer_pass.mlTraining = False


def getBRDF(pixel):
    path_tracer_pass.useMIS = False
    path_tracer_pass.indirectSky = gparams.enable_sky_learning
    path_tracer_pass.mlTraining = True
    primary_render_pass.brdfRender = True
    primary_render_pass.targetPixel = numpytouint2(pixel)
    testbed.frame()
    primary_render_pass.brdfRender = False
    path_tracer_pass.useMIS = True
    path_tracer_pass.indirectSky = True
    path_tracer_pass.mlTraining = False
    brdf = testbed.render_graph.get_output(primary_render_pass_name+".brdf").to_numpy()[:, :, :3]
    return brdf



In [11]:
#testbed.end_frame_forced()

In [ ]:
# feel free to remove it
scene_cfg.prepare_scene(ref_scene)

im = frameRender(num_samples = 1, vis=True, tonemapped=True)
# this should produce a converged render from the correct camera viewpoint (+- as in the paper teaser)

In [13]:
def falcor_to_torch(buffer: falcor.Buffer, dtype=torch.float32):
    params = torch.tensor([0]*(buffer.element_count*12), dtype=dtype)
    buffer.copy_to_torch(params)
    device.render_context.wait_for_cuda()
    return params

In [14]:
def extract_field(data, start, struct_size, field_size):
    return torch.cat([data[i:i + field_size] for i in range(start, data.numel(), struct_size)])

def falcor_to_torch_split_interleaved(buffer, field_types, dtype=torch.float32):
    # Size mapping for different types (add more types if needed)
    size_mapping = {
        "float3": 3,  # 3 floats in a float3
        "float": 1
    }

    # Calculate the size of one complete set of fields
    struct_size = sum(size_mapping[field_type] for field_type in field_types.values())
    # Calculate the total size of the tensor
    total_size = struct_size * buffer.element_count    
    all_data = buffer.to_torch([total_size])
    device.render_context.wait_for_cuda()

    # Splitting the tensor into separate tensors for each field considering interleaved structure
    tensors = {}
    offset = 0
    for i, (field_name, field_type) in enumerate(field_types.items()):
        field_size  = size_mapping[field_type]

        # Reshaping data
        reshaped_data = all_data.view(-1, struct_size)
        # Extracting field without using a loop
        tensors[field_name] = reshaped_data[:, offset: offset + field_size]
        offset += field_size

    return tensors


In [15]:
mlDataOutput = falcor_to_torch_split_interleaved(ml_data, field_types)
mlDataRaysOutput = falcor_to_torch_split_interleaved(ml_rays_data, rays_fields)

In [16]:
mlDataOutput_ref = {k: v.clone() for k, v in mlDataOutput.items()}
mlDataRaysOutput_ref = {k: v.clone() for k, v in mlDataRaysOutput.items()}

In [ ]:
# we gotta inform the validation points about the corresponding surface parameters (where we're gonna capute the hemispherical incident light visualization) 
#scene_cfg.add_validation_point([900/1280.0, 100/720.0])
scene_cfg.setup_validation_points(mlDataOutput_ref)
scene_cfg.prepare_scene(ref_scene, debugPointID=0)
im = frameRender(num_samples = 1, vis=True, tonemapped=True)

In [ ]:
scene_cfg.prepare_scene(ref_scene)

im = frameRender(num_samples = 1, vis=True, tonemapped=True)

In [ ]:
scene_cfg.prepare_scene(ref_scene)
_ = frameRender(num_samples=1, tonemapped=True)
setupBRDFCache()